In [119]:
import os
import glob
import numpy as np
import re
import matplotlib.pyplot as plt

# Find alle CSV-filer i mappen
curve_paths = glob.glob("../data/PC roteret/*.csv")

# Indlæs alle kurver
aligned_curves = [np.loadtxt(path, delimiter=",") for path in curve_paths]


def translate_filename(filename):
    """
    Oversætter 'Rør10Kurve3.csv' → 'Pipe 10 – Curve 3'
    """
    match = re.search(r"Rør(\d+)Kurve(\d+)", filename)
    if match:
        pipe_num = match.group(1)
        curve_num = match.group(2)
        return f"Pipe {pipe_num} – Curve {curve_num}"
    else:
        return filename  # fallback hvis ikke formatet matcher
labels = [translate_filename(os.path.basename(p).replace("_PC", "")) for p in curve_paths]
%matplotlib qt

# Længde

In [120]:
def curve_length(curve):
    return np.sum(np.linalg.norm(np.diff(curve, axis=0), axis=1))

lengths = [curve_length(curve) for curve in aligned_curves]

# Udskriv længder for hver kurve
for name, L in zip(labels, lengths):
    print(f"{name:<25} → Length: {L:.2f} mm")

lengths = np.array(lengths)

print("\n📏 Statistik over kurve-længder:")
print(f"Min:         {lengths.min():.2f}")
print(f"Maks:        {lengths.max():.2f}")
print(f"Gennemsnit:  {lengths.mean():.2f}")
print(f"Std. afvig:  {lengths.std():.2f}")



Pipe 01 – Curve 4         → Length: 249.57 mm
Pipe 01 – Curve 5         → Length: 253.51 mm
Pipe 02 – Curve 3         → Length: 226.69 mm
Pipe 02 – Curve 4         → Length: 296.75 mm
Pipe 03 – Curve 1         → Length: 211.98 mm
Pipe 03 – Curve 4         → Length: 175.71 mm
Pipe 04 – Curve 2         → Length: 385.41 mm
Pipe 05 – Curve 3         → Length: 222.99 mm
Pipe 05 – Curve 4         → Length: 212.66 mm
Pipe 05 – Curve 5         → Length: 253.18 mm
Pipe 06 – Curve 2         → Length: 261.29 mm
Pipe 06 – Curve 4         → Length: 261.16 mm
Pipe 06 – Curve 5         → Length: 232.53 mm
Pipe 07 – Curve 1         → Length: 246.99 mm
Pipe 07 – Curve 2         → Length: 259.30 mm
Pipe 07 – Curve 3         → Length: 282.04 mm
Pipe 07 – Curve 4         → Length: 231.21 mm
Pipe 08 – Curve 1         → Length: 215.46 mm
Pipe 08 – Curve 3         → Length: 279.47 mm
Pipe 08 – Curve 4         → Length: 282.62 mm
Pipe 08 – Curve 5         → Length: 244.41 mm
Pipe 09 – Curve 2         → Length

In [121]:
plt.figure(figsize=(12, 5))
plt.bar(labels, lengths, color='orchid')
plt.xticks(rotation=90)
plt.axhline(y=264.59, color='black', linestyle='--', linewidth=1.5, label='Mean length')
plt.ylabel("Length [mm]")
plt.title("Length of aligned 3D curves")
plt.tight_layout()
plt.show()


# Midter squiggle

In [122]:
def extract_middle_section(curve, fraction=0.16):
    """
    Returnerer den midterste del af kurven.
    fraction=0.3 → midterste 30%
    """
    n = len(curve)
    mid = n // 2
    half_window = int(n * fraction / 2)
    return curve[mid - half_window : mid + half_window]


### Kun midte plot

In [123]:
import matplotlib.cm as cm
# Plot-indstillinger
n = len(aligned_curves)
rows, cols = 4, 7  # Justér hvis du har flere/færre kurver

fig = plt.figure(figsize=(cols * 3, rows * 3))
n = len(aligned_curves)
cmap = cm.get_cmap('plasma', 40)  # alternativt 'tab10', 'Set1', 'turbo'  # eller 'nipy_spectral', 'viridis' osv.

for i, curve in enumerate(aligned_curves):
    ax = fig.add_subplot(rows, cols, i + 1, projection='3d')
    
    mid = extract_middle_section(curve, fraction=0.25)
    ax.plot(*mid.T, color=cmap(i), linewidth=1.5)
    ax.set_xticks([]); ax.set_yticks([]); ax.set_zticks([])
    ax.set_title(labels[i], fontsize=8)
    ax.set_xlabel("X", fontsize=6)
    ax.set_ylabel("Y", fontsize=6)
    ax.set_zlabel("Z", fontsize=6)
    ax.tick_params(labelsize=6)
    ax.view_init(elev=0, azim=90) 

plt.tight_layout()
plt.suptitle("Middle 'Squiggle' of Each Curve", fontsize=16, y=1.02)
plt.subplots_adjust(top=0.92)
plt.show()

C:\Users\lucca\AppData\Local\Temp\ipykernel_3136\1229060484.py:8: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma', 40)  # alternativt 'tab10', 'Set1', 'turbo'  # eller 'nipy_spectral', 'viridis' osv.


### Midte og lidt ryk plot

In [124]:
def extract_middle_section(curve, fraction=0.25, shift=0):
    """
    Returnerer midtersektionen af kurven, med valgfri forskydning.
    - fraction: hvor stor del af kurven du vil tage (som andel, fx 0.25)
    - shift: hvor mange punkter forskydes vinduet (positiv = højre, negativ = venstre)
    """
    n = len(curve)
    mid = n // 2 + shift
    half_window = int(n * fraction / 2)
    start = max(0, mid - half_window)
    end = min(n, mid + half_window)
    return curve[start:end]

# Dine manuelle justeringer
# Dine tilpasninger – nu i antal punkter (fx ±20 ud af 1000)
shifts = {
    3:  40,   # ryk lidt til højre (langs kurven)
    10: -40,
    22: -30,
    9: -30,  # ryk lidt til venstre
    20: 40
}


# Udtræk midterstykker med tilpasning (KUN forskel på dem du angiver)
centered_curves = []
for i, curve in enumerate(aligned_curves):
    shift = shifts.get(i, 0)
    section = extract_middle_section(curve, fraction=0.25, shift=shift)
    centered_curves.append(section)

fig = plt.figure(figsize=(cols * 3, rows * 3))
n = len(centered_curves)
cmap = cm.get_cmap('plasma', 40)

for i, curve in enumerate(centered_curves):
    ax = fig.add_subplot(rows, cols, i + 1, projection='3d')
    ax.plot(*curve.T, color=cmap(i), linewidth=1.5)
    ax.set_title(labels[i], fontsize=8)
    ax.set_xlabel("X", fontsize=6)
    ax.set_ylabel("Y", fontsize=6)
    ax.set_zlabel("Z", fontsize=6)
    ax.set_xticks([]); ax.set_yticks([]); ax.set_zticks([])

    ax.tick_params(labelsize=6)
    ax.view_init(elev=0, azim=270)

plt.tight_layout()
plt.suptitle("Middle 'Squiggle' of Each Curve (X-Z view)", fontsize=16, y=1.02)
plt.subplots_adjust(top=0.92)
plt.show()


C:\Users\lucca\AppData\Local\Temp\ipykernel_3136\257510618.py:34: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma', 40)


In [125]:
middle_lengths= []
amps = np.zeros((27,3))

for i, curve in enumerate(centered_curves):
    length = np.sum(np.linalg.norm(np.diff(centered_curves[i], axis=0), axis=1))
    middle_lengths.append(length)
    amp = np.ptp(centered_curves[i], axis=0)
    amps[i,:] = amp
    #print(f"{labels[i]} → Mid-length: {length:.2f} mm, Amplitude: {amp}")
print(sum(middle_lengths)/len(middle_lengths))
print(np.mean(amps, 0))

63.005867402727965
[19.76983377  1.77154364 23.47870128]


# Frekvens på Z-akse

In [126]:
from scipy.fft import fft, fftfreq
components = ['X', 'Y', 'Z']
plt.figure(figsize=(16, 10))

for j, axis in enumerate([0, 1, 2]):  # 0 = X, 1 = Y, 2 = Z
    plt.subplot(3, 1, j + 1)
    
    for i, curve in enumerate(aligned_curves):
        signal = curve[:, axis]
        N = len(signal)
        L = curve_length(curve)
        spacing = L / N

        F = fft(signal)
        freqs_mm = fftfreq(N, d=spacing)
        idx = (freqs_mm > 0) & (freqs_mm <= 0.1)  # zoom

        spectrum = 2.0 / N * np.abs(F[idx])
        plt.plot(freqs_mm[idx], spectrum, label=labels[i], alpha=0.4)

    plt.title(f"Frekvensspektrum af {components[j]}-komponent (0–0.1 [1/mm])")
    plt.xlabel("Frekvens [1/mm]")
    plt.ylabel("Amplitude")
    plt.grid(True)

plt.tight_layout()
plt.show()


In [127]:
print("Dominerende frekvens pr. komponent (i [1/mm]):\n")

for i, curve in enumerate(aligned_curves):
    dom_freqs = []
    N = len(curve)
    L = curve_length(curve)
    spacing = L / N

    for axis in range(3):
        signal = curve[:, axis]
        F = fft(signal)
        freqs_mm = fftfreq(N, d=spacing)
        idx = freqs_mm > 0
        dominant_freq = freqs_mm[idx][np.argmax(np.abs(F[idx]))]
        dom_freqs.append(dominant_freq)
    
    print(f"{labels[i]:<25} → X: {dom_freqs[0]:.4f}, Y: {dom_freqs[1]:.4f}, Z: {dom_freqs[2]:.4f}")


Dominerende frekvens pr. komponent (i [1/mm]):

Pipe 01 – Curve 4         → X: 0.0040, Y: 0.0040, Z: 0.0160
Pipe 01 – Curve 5         → X: 0.0039, Y: 0.0039, Z: 0.0039
Pipe 02 – Curve 3         → X: 0.0044, Y: 0.0044, Z: 0.0044
Pipe 02 – Curve 4         → X: 0.0034, Y: 0.0034, Z: 0.0135
Pipe 03 – Curve 1         → X: 0.0047, Y: 0.0047, Z: 0.0189
Pipe 03 – Curve 4         → X: 0.0057, Y: 0.0057, Z: 0.0057
Pipe 04 – Curve 2         → X: 0.0026, Y: 0.0026, Z: 0.0104
Pipe 05 – Curve 3         → X: 0.0045, Y: 0.0045, Z: 0.0045
Pipe 05 – Curve 4         → X: 0.0047, Y: 0.0047, Z: 0.0047
Pipe 05 – Curve 5         → X: 0.0039, Y: 0.0039, Z: 0.0158
Pipe 06 – Curve 2         → X: 0.0038, Y: 0.0038, Z: 0.0077
Pipe 06 – Curve 4         → X: 0.0038, Y: 0.0038, Z: 0.0038
Pipe 06 – Curve 5         → X: 0.0043, Y: 0.0043, Z: 0.0043
Pipe 07 – Curve 1         → X: 0.0040, Y: 0.0040, Z: 0.0040
Pipe 07 – Curve 2         → X: 0.0039, Y: 0.0039, Z: 0.0154
Pipe 07 – Curve 3         → X: 0.0035, Y: 0.0035, Z:

In [128]:
print(f"{'Kurve':<30} {'Frekvens [1/mm]':>18} {'Bølgelængde [mm]':>20} {'Kurvelængde [mm]':>20} {'Antal bølger':>15}")
print("-" * 105)

for i, curve in enumerate(aligned_curves):
    z = curve[:, 2]
    N = len(z)
    L = curve_length(curve)  # total længde i mm
    spacing = L / N

    Z = fft(z)
    freqs = fftfreq(N, d=spacing)
    idx = freqs > 0
    dominant_freq = freqs[idx][np.argmax(np.abs(Z[idx]))]
    wavelength = 1 / dominant_freq if dominant_freq > 0 else float('inf')
    num_waves = L / wavelength if wavelength > 0 else 0

    print(f"{labels[i]:<30} {dominant_freq:>18.4f} {wavelength:>20.2f} {L:>20.2f} {num_waves:>15.2f}")


Kurve                             Frekvens [1/mm]     Bølgelængde [mm]     Kurvelængde [mm]    Antal bølger
---------------------------------------------------------------------------------------------------------
Pipe 01 – Curve 4                          0.0160                62.39               249.57            4.00
Pipe 01 – Curve 5                          0.0039               253.51               253.51            1.00
Pipe 02 – Curve 3                          0.0044               226.69               226.69            1.00
Pipe 02 – Curve 4                          0.0135                74.19               296.75            4.00
Pipe 03 – Curve 1                          0.0189                53.00               211.98            4.00
Pipe 03 – Curve 4                          0.0057               175.71               175.71            1.00
Pipe 04 – Curve 2                          0.0104                96.35               385.41            4.00
Pipe 05 – Curve 3             

In [129]:
from scipy.signal import find_peaks

for i, curve in enumerate(aligned_curves):
    z = curve[:, 2]
    peaks, _ = find_peaks(z)
    valleys, _ = find_peaks(-z)
    num_waves = (len(peaks) + len(valleys)) / 2

    print(f"{labels[i]:<30} → Estimerede bølger i Z: {num_waves:.1f}")


Pipe 01 – Curve 4              → Estimerede bølger i Z: 6.5
Pipe 01 – Curve 5              → Estimerede bølger i Z: 5.5
Pipe 02 – Curve 3              → Estimerede bølger i Z: 4.5
Pipe 02 – Curve 4              → Estimerede bølger i Z: 4.5
Pipe 03 – Curve 1              → Estimerede bølger i Z: 4.5
Pipe 03 – Curve 4              → Estimerede bølger i Z: 3.5
Pipe 04 – Curve 2              → Estimerede bølger i Z: 3.5
Pipe 05 – Curve 3              → Estimerede bølger i Z: 3.5
Pipe 05 – Curve 4              → Estimerede bølger i Z: 3.5
Pipe 05 – Curve 5              → Estimerede bølger i Z: 4.5
Pipe 06 – Curve 2              → Estimerede bølger i Z: 3.5
Pipe 06 – Curve 4              → Estimerede bølger i Z: 3.5
Pipe 06 – Curve 5              → Estimerede bølger i Z: 3.5
Pipe 07 – Curve 1              → Estimerede bølger i Z: 3.5
Pipe 07 – Curve 2              → Estimerede bølger i Z: 4.5
Pipe 07 – Curve 3              → Estimerede bølger i Z: 3.5
Pipe 07 – Curve 4              → Estimer

# Lokal maks og min

In [130]:
from scipy.signal import find_peaks

fig = plt.figure(figsize=(cols * 3, rows * 3))

for i, curve in enumerate(aligned_curves):
    x, y, z = curve.T
    peaks, _ = find_peaks(z)
    valleys, _ = find_peaks(-z)

    ax = fig.add_subplot(rows, cols, i + 1, projection='3d')
    ax.plot(x, y, z, color='gray', linewidth=1.0)
    ax.scatter(x[peaks], y[peaks], z[peaks], color='red', marker='^', label='Toppe')
    ax.scatter(x[valleys], y[valleys], z[valleys], color='blue', marker='v', label='Dale')

    ax.set_title(labels[i], fontsize=8)
    ax.set_xlabel("X", fontsize=6)
    ax.set_ylabel("Y", fontsize=6)
    ax.set_zlabel("Z", fontsize=6)
    ax.tick_params(labelsize=6)
    ax.view_init(elev=0, azim=90)

# Justering
plt.tight_layout()
plt.suptitle("3D-visning af kurver med Z-toppe og -dale", fontsize=14, y=1.02)
plt.subplots_adjust(top=0.92)
plt.show()


# Gennemsnitskurve

### Lav den

In [131]:
from scipy.interpolate import interp1d

def reparameterize_by_arclength(curve, num_points=1000):
    # 1. Beregn kumulativ længde
    deltas = np.diff(curve, axis=0)
    segment_lengths = np.linalg.norm(deltas, axis=1)
    cumulative_length = np.insert(np.cumsum(segment_lengths), 0, 0)
    total_length = cumulative_length[-1]
    s = cumulative_length / total_length  # normaliseret arc length i [0,1]

    # 2. Interpolér X, Y, Z som funktion af s
    x_interp = interp1d(s, curve[:, 0], kind='linear')
    y_interp = interp1d(s, curve[:, 1], kind='linear')
    z_interp = interp1d(s, curve[:, 2], kind='linear')

    s_uniform = np.linspace(0, 1, num_points)
    reparam_curve = np.stack([
        x_interp(s_uniform),
        y_interp(s_uniform),
        z_interp(s_uniform)
    ], axis=1)

    return reparam_curve
# Reparameteriser alle kurver efter arc length
reparam_curves = [reparameterize_by_arclength(c, num_points=1000) for c in aligned_curves]

# Brug X fra den første kurve som reference
ref_x = reparam_curves[0][:, 0]

# Udtræk Y og Z fra alle kurver og tag gennemsnit
Y_all = np.stack([c[:, 1] for c in reparam_curves])
Z_all = np.stack([c[:, 2] for c in reparam_curves])

mean_y = np.mean(Y_all, axis=0)
mean_z = np.mean(Z_all, axis=0)

# Saml mean-kurven
mean_curve = np.stack([ref_x, mean_y, mean_z], axis=1)


### Suplots

In [132]:

fig = plt.figure(figsize=(cols * 3, rows * 3))
cmap = cm.get_cmap('plasma', 40)
for i, curve in enumerate(reparam_curves):
    ax = fig.add_subplot(rows, cols, i + 1, projection='3d')
    ax.plot(*curve.T, color = cmap(i), linewidth=1.0, alpha=0.7)
    ax.plot(*mean_curve.T, color='black', linewidth=2.0)
    
    ax.set_title(labels[i], fontsize=8)
    ax.tick_params(labelsize=6)
    ax.set_xlabel("X", fontsize=6)
    ax.set_ylabel("Y", fontsize=6)
    ax.set_zlabel("Z", fontsize=6)
    ax.view_init(elev=0, azim=270)

plt.tight_layout()
plt.suptitle("Kurver reparameteriseret efter arc length – med gennemsnit", fontsize=14, y=1.02)
plt.subplots_adjust(top=0.92)
plt.show()


C:\Users\lucca\AppData\Local\Temp\ipykernel_3136\3926769596.py:2: MatplotlibDeprecationWarning: The get_cmap function was deprecated in Matplotlib 3.7 and will be removed in 3.11. Use ``matplotlib.colormaps[name]`` or ``matplotlib.colormaps.get_cmap()`` or ``pyplot.get_cmap()`` instead.
  cmap = cm.get_cmap('plasma', 40)


### Alle i et

In [133]:
# Brug dine egne kurver i stedet for dummy data

# Plot alle rigtige kurver + mean
fig = plt.figure(figsize=(10, 6))
ax = fig.add_subplot(111, projection='3d')

for c in reparam_curves:
    ax.plot(c[:, 0], c[:, 1], c[:, 2], color='gray', alpha=0.3)

ax.plot(mean_curve[:, 0], mean_curve[:, 1], mean_curve[:, 2], color='black', linewidth=2, label='Mean Curve')
ax.set_xlabel("X")
ax.set_ylabel("Y")
ax.set_zlabel("Z")
ax.set_title("Dine egne kurver (gennemsigtige) + Mean Curve (sort)")
ax.legend()
plt.tight_layout()
plt.show()



### Stat

In [138]:
import pandas as pd
import numpy as np

# Statistik for mean-kurven
mean_diffs = np.diff(mean_curve, axis=0)
mean_lengths = np.linalg.norm(mean_diffs, axis=1)
mean_total_length = np.sum(mean_lengths)
mean_var = np.var(mean_curve, axis=0)
mean_std = np.std(mean_curve, axis=0)

# Saml i DataFrame
mean_stats = pd.DataFrame({
    "Coordinate": ["X", "Y", "Z"],
    "Mean": np.mean(mean_curve, axis=0),
    "Variance": mean_var,
    "Std Deviation": mean_std
})
mean_stats.loc[len(mean_stats)] = ["Total Length", mean_total_length, None, None]

# Print pænt
print("\nStatistik for mean-kurven:\n")
print(mean_stats.to_string(index=False))



Statistik for mean-kurven:

  Coordinate       Mean    Variance  Std Deviation
           X  -1.005650 1047.439635      32.364172
           Y   0.714548   54.667178       7.393726
           Z  -0.518607   84.331682       9.183228
Total Length 221.088390         NaN            NaN


C:\Users\lucca\AppData\Local\Temp\ipykernel_3136\1718283655.py:18: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  mean_stats.loc[len(mean_stats)] = ["Total Length", mean_total_length, None, None]


In [135]:
# Brug mean/std for X, Y og Z
X_all = np.stack([c[:, 0] for c in reparam_curves])
mean_x = np.mean(X_all, axis=0)
mean_y = np.mean(Y_all, axis=0)
mean_z = np.mean(Z_all, axis=0)

std_x = np.std(X_all, axis=0)
std_y = np.std(Y_all, axis=0)
std_z = np.std(Z_all, axis=0)


arc = np.linspace(0, 1, mean_x.shape[0])

# Plot X, Y, Z mean ± std
fig, axs = plt.subplots(3, 1, figsize=(10, 9), sharex=True)

# X
axs[0].plot(arc, mean_x, color='black', label='Mean X')
axs[0].fill_between(arc, mean_x - std_x, mean_x + std_x, color='orchid', alpha=0.3, label='±1 Std')
axs[0].set_ylabel("X")
axs[0].set_title("Mean ± Std Deviation")
axs[0].grid(True)
axs[0].legend()

# Y
axs[1].plot(arc, mean_y, color='black', label='Mean Y')
axs[1].fill_between(arc, mean_y - std_y, mean_y + std_y, color='orchid', alpha=0.3, label='±1 Std')
axs[1].set_ylabel("Y")
axs[1].grid(True)
axs[1].legend()

# Z
axs[2].plot(arc, mean_z, color='black', label='Mean Z')
axs[2].fill_between(arc, mean_z - std_z, mean_z + std_z, color='orchid', alpha=0.3, label='±1 Std')
axs[2].set_ylabel("Z")
axs[2].set_xlabel("Arc Length Position")
axs[2].grid(True)
axs[2].legend()

plt.tight_layout()
plt.show()


In [136]:
import matplotlib.pyplot as plt
import numpy as np

arc = np.linspace(0, 1, mean_z.shape[0])  # eller len(mean_z) hvis du er i tvivl

plt.figure(figsize=(10, 4))
plt.plot(arc, mean_z, color='black', label="Mean Z")
plt.fill_between(arc, mean_z - std_z, mean_z + std_z, color='gray', alpha=0.4, label="±1 Std Z")
plt.title("Mean Z with ±1 Std Deviation Band")
plt.xlabel("Arc Length Position")
plt.ylabel("Z")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
rms_errors = []
for c in reparam_curves:
    dy = c[:, 1] - mean_y
    dz = c[:, 2] - mean_z
    rms = np.sqrt(np.mean(dy**2 + dz**2))
    rms_errors.append(rms)

# Print eller brug som du vil
for i, rms in enumerate(rms_errors):
    print(f"{labels[i]:<25} → RMS-fejl til mean: {rms:.4f}")


Pipe 01 – Curve 4         → RMS-fejl til mean: 6.5332
Pipe 01 – Curve 5         → RMS-fejl til mean: 4.9474
Pipe 02 – Curve 3         → RMS-fejl til mean: 4.1256
Pipe 02 – Curve 4         → RMS-fejl til mean: 8.4481
Pipe 03 – Curve 1         → RMS-fejl til mean: 5.1799
Pipe 03 – Curve 4         → RMS-fejl til mean: 4.9438
Pipe 04 – Curve 2         → RMS-fejl til mean: 9.4553
Pipe 05 – Curve 3         → RMS-fejl til mean: 3.8530
Pipe 05 – Curve 4         → RMS-fejl til mean: 3.2740
Pipe 05 – Curve 5         → RMS-fejl til mean: 5.1202
Pipe 06 – Curve 2         → RMS-fejl til mean: 8.1547
Pipe 06 – Curve 4         → RMS-fejl til mean: 10.9126
Pipe 06 – Curve 5         → RMS-fejl til mean: 3.7247
Pipe 07 – Curve 1         → RMS-fejl til mean: 6.8355
Pipe 07 – Curve 2         → RMS-fejl til mean: 3.5427
Pipe 07 – Curve 3         → RMS-fejl til mean: 4.0879
Pipe 07 – Curve 4         → RMS-fejl til mean: 8.3482
Pipe 08 – Curve 1         → RMS-fejl til mean: 2.1758
Pipe 08 – Curve 3         →

In [139]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D


# Sort and select best and worst
indexed_curves = list(zip(labels, reparam_curves, rms_errors))
sorted_curves = sorted(indexed_curves, key=lambda x: x[2])
best = sorted_curves[:3]
worst = sorted_curves[-3:]

# Plot in 3D
fig = plt.figure(figsize=(15, 10))
titles = ["Best 1", "Best 2", "Best 3", "Worst 3", "Worst 2", "Worst 1"]

for i, (label, curve, _) in enumerate(best + worst[::-1]):
    ax = fig.add_subplot(2, 3, i+1, projection='3d')
    ax.plot(curve[:, 0], curve[:, 1], curve[:, 2], label='Original', linestyle='--', color = "orchid")
    ax.plot(mean_curve[:, 0], mean_curve[:, 1], mean_curve[:, 2], color='black', label='Mean')
    ax.set_title(f"{titles[i]}: {label}")
    ax.set_xticks([]); ax.set_yticks([]); ax.set_zticks([])
    ax.set_xlabel("X", fontsize=7)
    ax.set_ylabel("Y", fontsize=7)
    ax.set_zlabel("Z", fontsize=7)
    ax.view_init(elev=0, azim=270)
    ax.grid(True)

plt.tight_layout(pad=0.5)
plt.subplots_adjust(top=0.88)
plt.show()


# Næste